# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [56]:
city_data=pd.read_csv("cityweather_Output.csv",index_col=False)
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.1927,168.8643,8.04,97,42,1.48,NZ,1663156011
1,arraial do cabo,-22.9661,-42.0278,22.98,88,40,5.66,BR,1663155793
2,karmala,18.4167,75.2000,26.09,72,99,8.39,IN,1663155889
3,kruisfontein,-34.0033,24.7314,29.03,18,0,5.22,ZA,1663156012
4,saldanha,-33.0117,17.9442,18.06,57,0,6.17,ZA,1663156012


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [57]:
locations=city_data[["Lat","Lng"]].astype(float)


In [58]:
gmaps.configure(api_key=g_key)

In [59]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=city_data["Humidity"].astype(float), 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 6)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
city_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        564 non-null    object 
 1   Lat         564 non-null    float64
 2   Lng         564 non-null    float64
 3   Max Temp    564 non-null    float64
 4   Humidity    564 non-null    int64  
 5   Cloudiness  564 non-null    int64  
 6   Wind Speed  564 non-null    float64
 7   Country     558 non-null    object 
 8   Date        564 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 39.8+ KB


In [61]:
vacation_city=city_data.loc[(city_data["Max Temp"]<27)&(city_data["Max Temp"]>21)]

In [62]:
vacation_city=city_data.loc[(city_data["Max Temp"]<27) 
                           & (city_data["Max Temp"]>21) 
                           & (city_data["Wind Speed"] <4.5)
                            & (city_data["Cloudiness"]==0)]
vacation_city.head(20)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,podor,16.6527,-14.9587,24.97,83,0,0.00,MR,1663156031
74,port alfred,-33.5906,26.8910,21.21,76,0,4.02,ZA,1663155932
91,caravelas,-17.7125,-39.2481,25.87,54,0,4.48,BR,1663156059
106,knysna,-34.0363,23.0471,26.03,32,0,1.64,ZA,1663155865
107,lincoln,40.8000,-96.6670,21.18,63,0,2.68,US,1663156017
200,lake city,30.1897,-82.6393,22.57,88,0,0.00,US,1663156127
203,guerrero negro,27.9769,-114.0611,21.60,84,0,3.05,MX,1663156128
355,vila velha,-20.3297,-40.2925,24.97,72,0,4.12,BR,1663156200
374,nanakuli,21.3906,-158.1547,25.97,71,0,2.57,US,1663156237
474,manicore,-5.8092,-61.3003,22.97,100,0,1.03,BR,1663156305


In [63]:
vacation_city.shape

(11, 9)

In [64]:
vacation_clean_df=vacation_city.dropna()

In [65]:
vacation_clean_df.shape


(11, 9)

In [66]:
vacation_clean_df.head(6)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,podor,16.6527,-14.9587,24.97,83,0,0.00,MR,1663156031
74,port alfred,-33.5906,26.8910,21.21,76,0,4.02,ZA,1663155932
91,caravelas,-17.7125,-39.2481,25.87,54,0,4.48,BR,1663156059
106,knysna,-34.0363,23.0471,26.03,32,0,1.64,ZA,1663155865
107,lincoln,40.8000,-96.6670,21.18,63,0,2.68,US,1663156017
200,lake city,30.1897,-82.6393,22.57,88,0,0.00,US,1663156127


In [67]:
vacation_clean_df.reset_index(drop=True, inplace=True)


In [68]:
vacation_clean_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
vacation_clean_df["Hotel Name"] =""
vacation_clean_df
    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,podor,16.6527,-14.9587,24.97,83,0,0.00,MR,1663156031,
1,port alfred,-33.5906,26.8910,21.21,76,0,4.02,ZA,1663155932,
2,caravelas,-17.7125,-39.2481,25.87,54,0,4.48,BR,1663156059,
3,knysna,-34.0363,23.0471,26.03,32,0,1.64,ZA,1663155865,
4,lincoln,40.8000,-96.6670,21.18,63,0,2.68,US,1663156017,
5,lake city,30.1897,-82.6393,22.57,88,0,0.00,US,1663156127,
6,guerrero negro,27.9769,-114.0611,21.60,84,0,3.05,MX,1663156128,
7,vila velha,-20.3297,-40.2925,24.97,72,0,4.12,BR,1663156200,
8,nanakuli,21.3906,-158.1547,25.97,71,0,2.57,US,1663156237,
9,manicore,-5.8092,-61.3003,22.97,100,0,1.03,BR,1663156305,


In [70]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [71]:

# Use the lat/lng we recovered to identify hotels
import json


for index, row in vacation_clean_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url and convert to json
    name_hotel = requests.get(base_url, params=params).json()
    #print(name_hotel)
    #print(json.dumps(name_hotel, indent=4, sort_keys=True))
    
    try:
        vacation_clean_df.loc[index,"Hotel Name"]=name_hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
vacation_clean_df    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,podor,16.6527,-14.9587,24.97,83,0,0.00,MR,1663156031,Auberge Du Tékrour
1,port alfred,-33.5906,26.8910,21.21,76,0,4.02,ZA,1663155932,The Halyards Hotel
2,caravelas,-17.7125,-39.2481,25.87,54,0,4.48,BR,1663156059,Pousada dos Navegantes
3,knysna,-34.0363,23.0471,26.03,32,0,1.64,ZA,1663155865,Protea by Marriott Hotel Knysna Quays
4,lincoln,40.8000,-96.6670,21.18,63,0,2.68,US,1663156017,Courtyard by Marriott Lincoln Downtown/Haymarket
5,lake city,30.1897,-82.6393,22.57,88,0,0.00,US,1663156127,Ramada Limited Lake City
6,guerrero negro,27.9769,-114.0611,21.60,84,0,3.05,MX,1663156128,"Casa Laguna, Bed & Breakfast"
7,vila velha,-20.3297,-40.2925,24.97,72,0,4.12,BR,1663156200,Hotel Vitória Palace
8,nanakuli,21.3906,-158.1547,25.97,71,0,2.57,US,1663156237,Camp Pālehua
9,manicore,-5.8092,-61.3003,22.97,100,0,1.03,BR,1663156305,Hotel Durval


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_clean_df.iterrows()]
locations = vacation_clean_df[["Lat", "Lng"]]

In [73]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))